## Теоретическая часть
__1.__ Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?
В чем принципиальные отличия item-item рекомендаций от ALS?

В user-user ищутся товары по похожим пользователям, в item-item ищут по похожим товарам, в обоих алгоритмах используется алгоритм KNN. Не вижу функции item_iten_rec, сложно сказать, можно ли ее использовать, но если используется один алгоритм для нахождения похожих - то можно.
Принципиальное отличие item-item от ALS в том, что item-item алгоритмы основаны на определении расстояния между объектами, в ALS находят разложения матриц, которые приближают исходную матрицу.

__2.__ Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров

 - частота покупок за определенный период
 - сам факт покупки (бинарная матрица)
 - доля покупок товара среди всех купленных товаров
 - взвешенная метрика (например, количество просмотров * коэффициент + количество покупок * коэффициент)

__3.__ Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д

- не учитываются признаки пользователей/товаров
- не учитывается сезонность
- предсказывается не вероятность, а некое число (чем больше число, тем более релевантен товар)



In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [52]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [53]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [55]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [25]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 22.3 s, sys: 2.61 s, total: 24.9 s
Wall time: 10.9 s


In [26]:
[id_to_itemid[rec[0]] for rec in recs]

[1133018, 1106523, 999999, 5569230, 1082185]

In [27]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [28]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

CPU times: user 1min 23s, sys: 7.23 s, total: 1min 30s
Wall time: 46.8 s


0.16003917727717923

In [38]:
# подберем оптимальное число факторов

factors = [20, 25, 30, 40, 50, 120]
regularisations = [ 1e-3, 1e-2, 1e-1, 0, 1e1, 1e2]
iterations_ = [10, 15, 30, 50, 100]

for f in factors:
    for r in regularisations:
        for i in iterations_:
            
            model = AlternatingLeastSquares(factors=f, 
                                            regularization=r,
                                            iterations=i, 
                                            num_threads=4)

            model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=True)

            result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            res = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

            print(f"{30*'-'}\nЧисло факторов: {f}\nРегуляризация: {r}\nЧисло итераций: {i}\nPresision@5: {res}")

  0%|          | 0/10 [00:00<?, ?it/s]

------------------------------
Число факторов: 20
Регуляризация: 0.001
Число итераций: 10
Presision@5: 0.1644466209598433


  0%|          | 0/15 [00:00<?, ?it/s]

------------------------------
Число факторов: 20
Регуляризация: 0.001
Число итераций: 15
Presision@5: 0.16523016650342803


  0%|          | 0/30 [00:00<?, ?it/s]

...

------------------------------
Число факторов: 120
Регуляризация: 100.0
Число итераций: 30
Presision@5: 0.19618021547502448


  0%|          | 0/50 [00:00<?, ?it/s]

------------------------------
Число факторов: 120
Регуляризация: 100.0
Число итераций: 50
Presision@5: 0.19608227228207642


  0%|          | 0/100 [00:00<?, ?it/s]

------------------------------
Число факторов: 120
Регуляризация: 100.0
Число итераций: 100
Presision@5: 0.1958863858961802


Число факторов: 20
Регуляризация: 100.0
Число итераций: 100
Presision@5: 0.19578844270323212
    
Число факторов: 25
Регуляризация: 100.0
Число итераций: 30
Presision@5: 0.19618021547502448
    
Число факторов: 40
Регуляризация: 100.0
Число итераций: 50
Presision@5: 0.19618021547502448

In [56]:
# применим взвешивание bm25

user_item_matrix = bm25_weight(user_item_matrix.T).T 

In [47]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 1min 46s, sys: 10.4 s, total: 1min 56s
Wall time: 57.1 s


0.17071498530852106

In [57]:
# подберем оптимальные параметры для матрицы c взвешиванием bm25

factors = [20, 25, 30, 40, 50]
regularisations = [1e-2, 1e-1, 0, 1e1, 1e2]
iterations_ = [10, 15, 30, 50, 100]

for f in factors:
    for r in regularisations:
        for i in iterations_:
            
            model = AlternatingLeastSquares(factors=f, 
                                            regularization=r,
                                            iterations=i, 
                                            num_threads=4)

            model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=True)

            result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            res = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

            print(f"{30*'-'}\nЧисло факторов: {f}\nРегуляризация: {r}\nЧисло итераций: {i}\nPresision@5: {res}")

  0%|          | 0/10 [00:00<?, ?it/s]

------------------------------
Число факторов: 20
Регуляризация: 0.01
Число итераций: 10
Presision@5: 0.15788442703232128


  0%|          | 0/15 [00:00<?, ?it/s]

------------------------------
Число факторов: 20
Регуляризация: 0.01
Число итераций: 15
Presision@5: 0.16033300685602353


  0%|          | 0/30 [00:00<?, ?it/s]

...

------------------------------
Число факторов: 50
Регуляризация: 100.0
Число итераций: 30
Presision@5: 0.20127326150832517


  0%|          | 0/50 [00:00<?, ?it/s]

------------------------------
Число факторов: 50
Регуляризация: 100.0
Число итераций: 50
Presision@5: 0.20068560235063665


  0%|          | 0/100 [00:00<?, ?it/s]

------------------------------
Число факторов: 50
Регуляризация: 100.0
Число итераций: 100
Presision@5: 0.20117531831537708


In [58]:
# обучим лучшую модель

# Число факторов: 50
# Регуляризация: 100.0
# Число итераций: 30
# Presision@5: 0.20127326150832517


model = AlternatingLeastSquares(factors=50, 
                                regularization=100,
                                iterations=30, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['best_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['best_bm25'], row['actual']), axis=1).mean()

  0%|          | 0/30 [00:00<?, ?it/s]

0.2034280117531832

In [62]:
model = AlternatingLeastSquares(factors=100, 
                                regularization=100,
                                iterations=30, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['best_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['best_bm25'], row['actual']), axis=1).mean()

  0%|          | 0/30 [00:00<?, ?it/s]

0.21488736532810973

- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)

Можно сделать отдельную модель на акционные товары и взять топ 3, и на обычные товары настроить вторую модель и взять топ 2 или 7 (в зависимости от количества товаров для рекомендации)

- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)

Тоже можно сделать модель, отсортировав уже купленные товары и настроить вывод топа товаров среди новых.
Также можно при выводе взять не топ 5, например, а топ 10 и взять наиболее вероятный из новых продуктов (это можно сделать удалив из рекомендаций те, которые пользователь купил) и взять нужное количество рекомендованных товаров. Также в качестве фильтра собственных покупок можно использовать модель с одним соседом - в этом случае будут рекомендоваться его собственные покупки

- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Можно взять похожие товары из популярных категорий пользователя, но рекомендовать те, которые на определенный процент (например, на 15%, если мы рекомендуем 5 товаров) дороже среднего значения (посчитать среднее по всем товарам этой категории). Так же можно рекомендовать те же товары, которые пользователь покупает - но большего объема.